In [69]:
# coding: utf-8

# note: use python 2.7 (iw_dataset)
import sys
sys.path.append('../VQA/PythonHelperTools')
from vqaTools.vqa import VQA
import random
import os
import configs_py2 as configs
import pandas as pd
import json

In [70]:
# initialize VQA api for QA annotations
vqa=VQA(configs.annFile, configs.quesFile)

loading VQA annotations and questions into memory...
0:00:06.685289
creating index...
index created!


In [71]:
# load all annotations and store in a dataframe
annIds = vqa.getQuesIds()
anns = vqa.loadQA(annIds)
df=pd.DataFrame(anns)

In [72]:
# shuffle dataset randomly
df = df.sample(frac=1, random_state=configs.RANDOM_SEED).reset_index(drop=True)

#filter dataset as necessary
# only consider non yes/no and non-number answer type
df = df[df.answer_type=="other"]

In [73]:
# randomly permute question ids and store in a new column
random_question_id = df.question_id.tolist()
random.seed(configs.RANDOM_SEED)
random.shuffle(random_question_id)
df['random_question_id'] = random_question_id

In [74]:
# get questions corresponding to the random question ids
random_question = [vqa.qqa[quesId]['question'] for quesId in random_question_id]
random_question = [item.encode('UTF8') for item in random_question]

# get questions corresponding to the original question ids
original_question_id = df.question_id.tolist()
original_question = [vqa.qqa[quesId]['question'] for quesId in original_question_id]
original_question = [item.encode('UTF8') for item in original_question]

In [75]:
#store original and random questions in a new column in the dataframe
df['random_question'] = random_question
df['original_question'] = original_question

In [76]:
df.head()

,answer_type,answers,image_id,multiple_choice_answer,question_id,question_type,random_question_id,random_question,original_question
3,other,"[{u'answer': u'hat', u'answer_confidence': u'y...",264853,visor,264853000,what is,105335001,What food is cooked?,What is covering the man's eyes?
5,other,"[{u'answer': u'stay in your lane', u'answer_co...",526401,wear helmet,526401003,none of the above,405047001,What color is the outline of the sign painted ...,Something you would say to give advice to anyo...
12,other,"[{u'answer': u'flying kite', u'answer_confiden...",551910,flying kite,551910010,what is this,450500003,What color shirt is the most common?,What is this person doing?
13,other,"[{u'answer': u'teddy bear', u'answer_confidenc...",195406,teddy bear,195406002,what kind of,202886000,What side is the broccoli place on?,What kind of toy is this?
17,other,"[{u'answer': u'green', u'answer_confidence': u...",138382,green,138382006,what color is the,11156002,What is the main color of the bag?,What Color is the vegetable?


In [77]:
# store updated dataframe in a csv file
if not os.path.exists(configs.experimentDatasetDir):
   os.makedirs(configs.experimentDatasetDir)
dataset_path = "%s/%s%s_%s_%s.csv" % (configs.experimentDatasetDir, configs.versionType, configs.taskType, configs.dataType, configs.dataSubType)
df.to_csv(dataset_path, index=False)